In [197]:
import sklearn as sk
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # 모든 경고 메시지 무시
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'DejaVu Sans'  # 영어 전용 폰트 사용



# matplotlib에서 한글 폰트 설정

# plt.rcParams['font.family'] = 'NanumGothic'  # 나눔고딕 폰트 사용
# plt.rcParams['font.family'] = 'Malgun Gothic'  # 맑은 고딕
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac용 폰트

# KMeans 경고 메시지 제거를 위한 파라미터 명시적 설정
from sklearn.cluster import KMeans

In [198]:
from sklearn import datasets

customerSeg = pd.read_csv('/apps/study_machinelearnings/datasets/Train.csv')

### 전처리
    + 컬럼 삭제
    + 결측치 채워넣기
    + OneHotEncoder 사용해서 수치데이터가 아닌 데이터 수정

In [199]:
customerSeg.drop(['ID', 'Var_1', 'Segmentation', 'Profession'], axis=1, inplace=True)

# customerSeg['Profession'].value_counts()
# customerSeg['Family_Size'].value_counts()

In [200]:
# 1. 기본적인 결측치 개수 확인
customerSeg.isnull().sum()

# 2. 결측치 비율(%) 확인
(customerSeg.isnull().sum() / len(customerSeg) * 100).round(2)

# 3. 개수와 비율을 동시에 데이터프레임으로 확인
pd.DataFrame({
    '결측치 개수': customerSeg.isnull().sum(),
    '결측치 비율(%)': (customerSeg.isnull().sum() / len(customerSeg) * 100).round(2)
})


,결측치 개수,결측치 비율(%)
Gender,0,0.00
Ever_Married,140,1.74
Age,0,0.00
Graduated,78,0.97
Work_Experience,829,10.28
Spending_Score,0,0.00
Family_Size,335,4.15


In [201]:
# 컬럼별로 결측치 채우기(대표값 Or 0)

# Work_Experience - 3살 단위 그룹의 평균
customerSeg['Age_group_3'] = (customerSeg['Age'] // 3) * 3
work_exp_mean = customerSeg.groupby('Age_group_3')['Work_Experience'].transform('mean')
customerSeg['Work_Experience'] = customerSeg['Work_Experience'].fillna(work_exp_mean)
customerSeg.drop('Age_group_3', axis=1, inplace=True)

# Ever_Married - 5살 단위 그룹의 최빈값(mode)
customerSeg['Age_group_5'] = (customerSeg['Age'] // 5) * 5
# 각 나이 그룹별 최빈값 구하기
married_mode = customerSeg.groupby('Age_group_5')['Ever_Married'].apply(lambda x: x.mode().iloc[0])

# 결측치를 해당 나이 그룹의 최빈값으로 채우기
for age_group in married_mode.index:
    mask = (customerSeg['Age_group_5'] == age_group) & (customerSeg['Ever_Married'].isnull())
    customerSeg.loc[mask, 'Ever_Married'] = married_mode[age_group]
customerSeg.drop('Age_group_5', axis=1, inplace=True)

# Family_size - 0으로 채우기
customerSeg['Family_Size'] = customerSeg['Family_Size'].fillna(0)

# Graduated - 10살 단위 그룹의 최빈값(mode)
customerSeg['Age_group_10'] = (customerSeg['Age'] // 10) * 10
# 각 나이 그룹별 최빈값 구하기
graduated_mode = customerSeg.groupby('Age_group_10')['Graduated'].apply(lambda x: x.mode().iloc[0])

# 결측치를 해당 나이 그룹의 최빈값으로 채우기
for age_group in graduated_mode.index:
    mask = (customerSeg['Age_group_10'] == age_group) & (customerSeg['Graduated'].isnull())
    customerSeg.loc[mask, 'Graduated'] = graduated_mode[age_group]
customerSeg.drop('Age_group_10', axis=1, inplace=True)


In [202]:
# 결측치 처리 결과 확인
print(customerSeg[['Work_Experience', 'Ever_Married', 'Family_Size', 'Graduated']].isnull().sum())

# Ever_Married와 Graduated의 값이 Yes/No로만 구성되어 있는지 확인
print("\nEver_Married 고유값:", customerSeg['Ever_Married'].unique())
print("Graduated 고유값:", customerSeg['Graduated'].unique())

# customerSeg

Work_Experience    0
Ever_Married       0
Family_Size        0
Graduated          0
dtype: int64

Ever_Married 고유값: ['No' 'Yes']
Graduated 고유값: ['No' 'Yes']


In [203]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

# 범주형 변수 선택
categorical_columns = ['Gender', 'Ever_Married', 'Graduated', 'Spending_Score']

# OneHotEncoder 생성
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# 범주형 변수에 대해 원핫인코딩 수행
encoded_data = encoder.fit_transform(customerSeg[categorical_columns])

# 인코딩된 열 이름 생성
encoded_feature_names = []
for i, column in enumerate(categorical_columns):
    categories = encoder.categories_[i]
    for category in categories:
        encoded_feature_names.append(f'{column}_{category}')

# 인코딩된 데이터를 데이터프레임으로 변환
encoded_df = pd.DataFrame(encoded_data, columns=encoded_feature_names)

# 기존 데이터프레임에서 범주형 변수 제거
customerSeg_numeric = customerSeg.drop(categorical_columns, axis=1)

# 인코딩된 데이터와 나머지 데이터 결합
customerSeg_encoded = pd.concat([customerSeg_numeric, encoded_df], axis=1)

# 결과 확인
print("원본 데이터 형태:", customerSeg.shape)
print("변환된 데이터 형태:", customerSeg_encoded.shape)
print("\n변환된 열 목록:")
print(customerSeg_encoded.columns.tolist())

# 변환된 데이터의 처음 몇 행 확인
print("\n변환된 데이터 미리보기:")
print(customerSeg_encoded.head())

# 각 범주형 변수의 원핫인코딩 결과 확인
for column in categorical_columns:
    print(f"\n{column} 인코딩 결과:")
    encoded_cols = [col for col in encoded_df.columns if col.startswith(column)]
    print(encoded_df[encoded_cols].head())


원본 데이터 형태: (8068, 7)
변환된 데이터 형태: (8068, 12)

변환된 열 목록:
['Age', 'Work_Experience', 'Family_Size', 'Gender_Female', 'Gender_Male', 'Ever_Married_No', 'Ever_Married_Yes', 'Graduated_No', 'Graduated_Yes', 'Spending_Score_Average', 'Spending_Score_High', 'Spending_Score_Low']

변환된 데이터 미리보기:
   Age  Work_Experience  Family_Size  Gender_Female  Gender_Male  \
0   22         1.000000          4.0            0.0          1.0   
1   38         3.588435          3.0            1.0          0.0   
2   67         1.000000          1.0            1.0          0.0   
3   67         0.000000          2.0            0.0          1.0   
4   40         3.523891          6.0            1.0          0.0   

   Ever_Married_No  Ever_Married_Yes  Graduated_No  Graduated_Yes  \
0              1.0               0.0           1.0            0.0   
1              0.0               1.0           0.0            1.0   
2              0.0               1.0           0.0            1.0   
3              0.0          